## 引入套件

In [1]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Dropout, Input
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints

C:\Users\user\Anaconda3\envs\keras\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 任務：文本情感分析
### 資料：imdb電影評論 
### 格式：文本內容 、 正負評

In [2]:
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words) #只取前5000個常用字

max_review_length = 500 #限制文本長度
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length) #強制補齊文本長度
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
embedding_vecor_length = 32

## 實現Attention

In [3]:
#自訂keras layer的教學 https://keras.io/layers/writing-your-own-keras-layers/

#將attention實現為一 layer
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs): #**kwargs https://eastlakeside.gitbooks.io/interpy-zh/content/args_kwargs/Using_args_and_kwargs_to_call_function.html
        """
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
        """
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        # 創建一變數儲存可訓練的權重
        assert len(input_shape) == 3
        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim
        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))
        if self.bias:
            eij += self.b
            
        eij = K.tanh(eij)
        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)
        weighted_input = x * a
        
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

## 建立模型
#### LSTM + Attention

In [8]:
comment_input = Input(shape=(max_review_length, ), dtype='int32')
embedded_sequences= Embedding(top_words, embedding_vecor_length, input_length=max_review_length)(comment_input)  
x = LSTM(100,return_sequences=True)(embedded_sequences)
x = Attention(max_review_length)(x)
preds = Dense(1, activation='sigmoid')(x)


model = Model(inputs=[comment_input],outputs=preds)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
#訓練模型
model.fit(X_train, y_train, epochs=3, batch_size=64)

# 驗證模型效果
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/3
25000/25000 [==============================] - 170s 7ms/step - loss: 0.4704 - acc: 0.7625
Epoch 2/3
25000/25000 [==============================] - 165s 7ms/step - loss: 0.2633 - acc: 0.8954
Epoch 3/3
25000/25000 [==============================] - 166s 7ms/step - loss: 0.2146 - acc: 0.9167
Accuracy: 87.76%


#### LSTM

In [5]:
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])
# 訓練模型
model.fit(X_train, y_train, epochs=3, batch_size=64)

# 驗證模型效果
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/3
25000/25000 [==============================] - 176s 7ms/step - loss: 0.5056 - acc: 0.7486
Epoch 2/3
25000/25000 [==============================] - 175s 7ms/step - loss: 0.3894 - acc: 0.8316
Epoch 3/3
25000/25000 [==============================] - 174s 7ms/step - loss: 0.3448 - acc: 0.8562
Accuracy: 82.60%
